In [10]:
import numpy as np
import pandas as pd
from keras import models
from keras import layers
np.random.seed(7)

## Import the data

In [16]:
data = import_data()

## Separate test set and normalize the data

In [17]:
num_test_samples = 47 # 25% of all samples

np.random.shuffle(data)
test_data = data[:num_test_samples]
data = data[num_test_samples:]

mean = data[:,:-6].mean(axis=0) # leave the target variables out of normalization!
data[:,:-6] -= mean
std = data[:,:-6].std(axis=0)
data[:,:-6] /= std

test_data[:,:-6] -= mean
test_data[:,:-6] /= std

## Define model
### example of hyperparameter search

In [13]:
layer1_sizes = [8, 16, 32, 64] # hidden layer sizes to search through
layer2_sizes = [8, 16, 32, 32]

def build_model(layer1_size, layer2_size):
    model = models.Sequential()
    model.add(layers.Dense(layer1_size, activation='relu', 
                           input_shape=(train_X.shape[1],)))
    model.add(layers.Dense(layer2_size, activation='relu'))
    model.add(layers.Dense(6))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

## Train

In [14]:
k = 3
num_val_samples = len(data) // k
num_epochs = 500

validation_scores_all = []

for layer1_size, layer2_size in zip(layer1_sizes, layer2_sizes):
    print('layer1_size= %i, layer2_size= %i' % (layer1_size, layer2_size))
    validation_scores = []
    
    for fold in range(k):
        validation_data = data[num_val_samples * fold:
                               num_val_samples *(fold+1)]
        training_data = np.concatenate([data[:num_val_samples * fold],
                                        data[num_val_samples * (fold+1):]], axis=0)

        train_X = training_data[:,:-6]
        train_y = training_data[:,-6:]
        val_X = validation_data[:,:-6]
        val_y = validation_data[:,-6:]

        print('processing fold #', fold)
        model = build_model(layer1_size, layer2_size)
        train_history = model.fit(train_X, train_y, epochs=num_epochs, 
                                  validation_data=(val_X, val_y), 
                                  batch_size=1, verbose=0)

        validation_score = model.evaluate(val_X, val_y, verbose=0)
        validation_scores.append(validation_score)
        
    validation_scores_all.append(validation_scores)

layer1_size= 8, layer2_size= 8
processing fold # 0
processing fold # 1
processing fold # 2
layer1_size= 16, layer2_size= 16
processing fold # 0
processing fold # 1
processing fold # 2
layer1_size= 32, layer2_size= 32
processing fold # 0
processing fold # 1
processing fold # 2
layer1_size= 64, layer2_size= 32
processing fold # 0
processing fold # 1
processing fold # 2


In [15]:
for i, layer_sizes in enumerate(zip(layer1_sizes, layer2_sizes)):
    print('hidden layer sizes: %s \n average val_mse: %2f average val_mae: %2f' % (layer_sizes, np.average(validation_scores_all[i], axis=0)[0], np.average(validation_scores_all[i], axis=0)[1]))

hidden layer sizes: (8, 8) 
 average val_mse: 42.738263 average val_mae: 3.966958
hidden layer sizes: (16, 16) 
 average val_mse: 39.412308 average val_mae: 3.884672
hidden layer sizes: (32, 32) 
 average val_mse: 37.539061 average val_mae: 3.431678
hidden layer sizes: (64, 32) 
 average val_mse: 31.151189 average val_mae: 3.339603
